# <font color='darkgreen'>Applied data science capstone- final assignment project</font>

---

## <font color='green'>Introduction</font>

Traffic accidents have an adverse effect both on an individual and societal level resulting in costs due to person injuries and property damage, increased travel times and emissions due to congestion.

Although the outcome of one accident can be very different to another, the underlying cause can be very similar. Roads shrouded in dense fog can make driving incredibly dangerous, limiting visibility of the looming road ahead. Large amounts of rain or snowfall can also provide treacherous driving conditions which will affect a great number of road users.

The question is:

### <i>Given the weather and the road conditions- what is the possibility of a person getting into a car accident and how severe it would be, so that he/she would drive more carefully or even change his/her travel if he/she is able to.</i>

#### Target audience for the presented research will be all of the people with the driving license.

---

## <font color='green'>Data description</font>

#### High level description

The dataset covers all types collisions (events gathered from 2004 till now) provided with such information as severity of the accident, weather or an indication, that the accident involved hitting the parked car or whether the driver was under the influence of drug or alcohol (to distingush accidents that not neccessarily have to be connected with weather conditions).

#### Features details

<b>SEVERITYCODE</b>- a code that corresponds to the severity of the collision: 3-fatality;2b-serious injury;2-injury;1-prop damage;0-unknown<br>
<b>UNDERINFL</b>- whether or not a driver involved was under the influence of drugs or alcohol<br>
<b>WEATHER</b>- a description of the weather conditions during the time of the collision<br>
<b>ROADCOND</b>- the condition of the road during the collision<br>
<b>LIGHTCOND</b>- the light conditions during the collision<br>
<b>SPEEDING</b>- whether or not speeding was a factor in the collision<br>
<b>HITPARKEDCAR</b>- whether or not the collision involved hitting a parked car<br>
<b>PERSONCOUNT</b>- the total number of people involved in the collision<br>
<b>INATTENTIONIND</b>- whether or not collision was due to innatention

#### Example

In [58]:
coll_df[4:5]

,WEATHER,SEVERITYCODE,ROADCOND,LIGHTCOND,SPEEDING,HITPARKEDCAR,UNDERINFL,PERSONCOUNT,INATTENTIONIND
4,Raining,2,Wet,Daylight,NaN,N,0,2,NaN


Example shown above illustrates the incident resulting with injuries. No parked vehicles were taking part in the event and speeding or drugs/alcohol were not the reason. Incident occured in the daylight, but probably wet road conditions and rainy weather were the root cause.

---

## <font color='green'>Data preparation</font>

### Importing necessary modules

In [4]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt

### Reading the input dataset

In [22]:
# The code was removed by Watson Studio for sharing.

In [23]:
fullcoll_df = pd.read_csv(body)
fullcoll_df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


### Picking needed columns

In [43]:
coll_df=fullcoll_df[['WEATHER','SEVERITYCODE','ROADCOND','LIGHTCOND','SPEEDING','HITPARKEDCAR','UNDERINFL','PERSONCOUNT','INATTENTIONIND']]
coll_df.head()

,WEATHER,SEVERITYCODE,ROADCOND,LIGHTCOND,SPEEDING,HITPARKEDCAR,UNDERINFL,PERSONCOUNT,INATTENTIONIND
0,Overcast,2,Wet,Daylight,NaN,N,N,2,NaN
1,Raining,1,Wet,Dark - Street Lights On,NaN,N,0,2,NaN
2,Overcast,1,Dry,Daylight,NaN,N,0,4,NaN
3,Clear,1,Dry,Daylight,NaN,N,N,3,NaN
4,Raining,2,Wet,Daylight,NaN,N,0,2,NaN


### Counting null values

In [44]:
coll_df.isnull().sum()

WEATHER             5081
SEVERITYCODE           0
ROADCOND            5012
LIGHTCOND           5170
SPEEDING          185340
HITPARKEDCAR           0
UNDERINFL           4884
PERSONCOUNT            0
INATTENTIONIND    164868
dtype: int64

### Results show most NULL values in the Speeding and Inattentionind columns, but it is proven below, that these should be replaced by 'N' (as the only remaining value is 'Y')

In [45]:
print('Unique values for WEATHER:\n%s\n'% coll_df['WEATHER'].unique())
print('Unique values for SEVERITYCODE:\n%s\n'% coll_df['SEVERITYCODE'].unique())
print('Unique values for ROADCOND:\n%s\n'% coll_df['ROADCOND'].unique())
print('Unique values for LIGHTCOND:\n%s\n'% coll_df['LIGHTCOND'].unique())
print('Unique values for SPEEDING:\n%s\n'% coll_df['SPEEDING'].unique())
print('Unique values for HITPARKEDCAR:\n%s\n'% coll_df['HITPARKEDCAR'].unique())
print('Unique values for UNDERINFL:\n%s\n'% coll_df['UNDERINFL'].unique())
print('Unique values for INATTENTIONIND:\n%s\n'% coll_df['INATTENTIONIND'].unique())

Unique values for WEATHER:
['Overcast' 'Raining' 'Clear' nan 'Unknown' 'Other' 'Snowing'
 'Fog/Smog/Smoke' 'Sleet/Hail/Freezing Rain' 'Blowing Sand/Dirt'
 'Severe Crosswind' 'Partly Cloudy']

Unique values for SEVERITYCODE:
[2 1]

Unique values for ROADCOND:
['Wet' 'Dry' nan 'Unknown' 'Snow/Slush' 'Ice' 'Other' 'Sand/Mud/Dirt'
 'Standing Water' 'Oil']

Unique values for LIGHTCOND:
['Daylight' 'Dark - Street Lights On' 'Dark - No Street Lights' nan
 'Unknown' 'Dusk' 'Dawn' 'Dark - Street Lights Off' 'Other'
 'Dark - Unknown Lighting']

Unique values for SPEEDING:
[nan 'Y']

Unique values for HITPARKEDCAR:
['N' 'Y']

Unique values for UNDERINFL:
['N' '0' nan '1' 'Y']

Unique values for INATTENTIONIND:
[nan 'Y']



### After finding unique labels, when it comes to the severity, we are dealing only with injuries or property damage. We need to remove 'Other' road conditions as they are not informative enough. The same goes for 'nan' and 'Óther' light conditions and 'nan' and 'Unknown' in the 'Weather' column. In the column 'Speeding', we are dealing only with 'nan' or 'Y'- therefore we can assume, that 'nan' can be replaced by 'N'- we need to treatthe data from 'INATTENTIONIND' analogically. In the column 'Underinfl' we need to remove 'nan' values, replace 0 with 'N' and 1 with 'Y'.

# zera usunac lub zamienic na dane, przeformatowac kolumny na liczby i labelki